### Structure 
* hypterparameters
* Define autonomous driving scenario environment
* Define replay buffer
* Define neural network
    * Define actor network
    * Define critic network
* Define RL framework
* Define training loop
    * select action, get feedback
    * store experience
    * learn using a minibatch

In [2]:
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
from collections import namedtuple

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# hypterparameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
BUFFER_CAPACITY = 2000      # capacity of replay buffer
steps_done = 0
RENDER = False # whether to render the environment or not

# Define autonomous driving scenario environment
class ADSEnv(object):
    def __init__(self):
        pass
        
    def step(self, action):
        """return next observation, reward, done, info"""
        pass
        
    def reset(self):
        pass
        
    def render(self):
        pass
        
    def close(self):
        pass

# Define replay buffer
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'done'))

class ReplayBuffer(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.replay_buffer = []
        self.position = 0

    def store(self, *args):
        """Saves a transition experience."""
        if len(self.replay_buffer) < self.capacity:
            self.replay_buffer.append(None)
        self.replay_buffer[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.replay_buffer, batch_size)

    def __len__(self):
        return len(self.replay_buffer)

# Define neural networks
class ActorNetwork(nn.Module):
    def __init__(self, ):
        super().__init__()

    def forward(self, x):
        pass
    
class CriticNetwork(nn.Module):
    def __init__(self, ):
        super().__init__()

    def forward(self, x):
        pass
    
# Define DDPG framework
class DDPG(object):
    def __init__(self):
        self.eval_net, self.target_net = CriticNetwork().to(device), CriticNetwork().to(device)
        self.policy_net = ActorNetwork().to(device)
        self.learn_step_counter = 0     # counter to update target network
        self.replay_buffer = ReplayBuffer(BUFFER_CAPACITY)
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)        
        self.loss_func = nn.MSELoss()   
    
    def choose_action(self, x):
        """select an action based on the current observation"""
        pass
        
    def learn(self):
        """update actor and critic network"""
        pass

# training loop
env = ADSEnv()
rl = DDPG() # initiate the RL framework

for i_episode in range(1000):
    s = env.reset()
    time_count = 1
    while True:
        if RENDAR:
            env.render()    # show environment
        a = rl.choose_action(s)

        # select action and get feedback
        s_, r, done, info = env.step(a)

        # store transition experience into replay buffer
        rl.replay_buffer.store(s, a, r, s_, done)
        
        # learning
        rl.learn()
        
        # episode terminates, enter next episode
        if done:                
            break

        s = s_
        time_count += 1
    if i_episode % 50 == 0:
        print("Episode finished after {} timesteps".format(time_count+1))
env.close()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 68)